In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

In [4]:
df = pd.read_csv("gender_classification.csv")

In [5]:
df.head()

,Sentences,label
0,"Though not originally on call, Nathan stepped ...",1
1,A mother recounted that balancing full-time wo...,0
2,The domestic skills of the women in the villag...,0
3,The unitary government structure centralized p...,0
4,"Although he may seem innocent and naïve, the b...",1


In [6]:
len(df)

11061

In [7]:
dataset = Dataset.from_pandas(df)

In [8]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)  # 70% train, 30% temp
test_valid = dataset["test"].train_test_split(test_size=0.5, seed=42)  # split into val/test
dataset["validation"] = test_valid["train"]
dataset["test"] = test_valid["test"]

In [9]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["Sentences"], padding="max_length", truncation=True, max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/8848 [00:00<?, ? examples/s]

Map:   0%|          | 0/1107 [00:00<?, ? examples/s]

Map:   0%|          | 0/1106 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels),
        "f1": f1.compute(predictions=preds, references=labels, average="weighted"),
    }

In [11]:
training_args = TrainingArguments(
    output_dir="./roberta_gender_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3298610078.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.098200,0.092031,{'accuracy': 0.972875226039783},{'f1': 0.9728592394142649}
2,0.029500,0.100614,{'accuracy': 0.9737793851717902},{'f1': 0.9737674751674857}
3,0.014400,0.098524,{'accuracy': 0.969258589511754},{'f1': 0.9692505407842367}


TrainOutput(global_step=1659, training_loss=0.08309575926115026, metrics={'train_runtime': 657.8592, 'train_samples_per_second': 40.349, 'train_steps_per_second': 2.522, 'total_flos': 1746004963368960.0, 'train_loss': 0.08309575926115026, 'epoch': 3.0})

In [15]:
metrics = trainer.evaluate(tokenized["test"])
print(metrics)

{'eval_loss': 0.07634978741407394, 'eval_accuracy': {'accuracy': 0.981029810298103}, 'eval_f1': {'f1': 0.9810308940851079}, 'eval_runtime': 7.8373, 'eval_samples_per_second': 141.247, 'eval_steps_per_second': 8.932, 'epoch': 3.0}


In [16]:
!pip install huggingface_hub

In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
trainer.save_model("./roberta_gender_classifier")
tokenizer.save_pretrained("./roberta_gender_classifier")

('./roberta_gender_classifier/tokenizer_config.json',
 './roberta_gender_classifier/special_tokens_map.json',
 './roberta_gender_classifier/vocab.json',
 './roberta_gender_classifier/merges.txt',
 './roberta_gender_classifier/added_tokens.json',
 './roberta_gender_classifier/tokenizer.json')

In [19]:
from huggingface_hub import create_repo

repo_id = "moatazhamza194/gender_classification-roberta"
create_repo(repo_id, private=False)   # set private=True if you don’t want it public

RepoUrl('https://huggingface.co/moatazhamza194/gender_classification-roberta', endpoint='https://huggingface.co', repo_type='model', repo_id='moatazhamza194/gender_classification-roberta')

In [20]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id=repo_id,
    folder_path="./roberta_gender_classifier",
    commit_message="Initial commit of fine-tuned RoBERTa model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ssifier/model.safetensors:   0%|          |  549kB /  499MB            

  ...ssifier/training_args.bin:   2%|1         |   108B / 5.71kB            

CommitInfo(commit_url='https://huggingface.co/moatazhamza194/gender_classification-roberta/commit/1c4593cf8d64aad1d715fbc2ebea1c31a163f50c', commit_message='Initial commit of fine-tuned RoBERTa model', commit_description='', oid='1c4593cf8d64aad1d715fbc2ebea1c31a163f50c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/moatazhamza194/gender_classification-roberta', endpoint='https://huggingface.co', repo_type='model', repo_id='moatazhamza194/gender_classification-roberta'), pr_revision=None, pr_num=None)

In [23]:
from transformers import pipeline

# Directly load from Hugging Face Hub
classifier = pipeline(
    "text-classification",
    model="moatazhamza194/gender_classification-roberta",
    tokenizer="moatazhamza194/gender_classification-roberta"
)


Device set to use cuda:0


In [27]:
result = classifier("All these guys I used to think were brilliant, then I gradually realized they are just the other side of the same coin: basically secular fundamentalists. Seeing Hitchens argue that male circumcision is exactly the same as female circumcision was final straw for me.")
print(result)

[{'label': 'LABEL_1', 'score': 0.9989005327224731}]
